In [92]:
from pandas import DataFrame
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential,load_model,Model
from keras.layers import LSTM,Dense,Input,Flatten,Concatenate
from keras.layers import LSTMCell,RNN
from math import sqrt
from matplotlib import pyplot
import numpy as np
import pandas as pd
import tensorflow as tf
#sess =tf.Session()

In [93]:
raw_data=np.loadtxt('./TapData/data_raw_v2.csv',delimiter=",")
y=np.loadtxt('./TapData/data_y_active.csv',delimiter=",")

In [4]:
#raw_data=np.loadtxt('./TapData/WatchData (36)_raw.csv',delimiter=",")

In [95]:
#raw_data=raw_data[:,0:6]
num_inputs=6
raw_data=raw_data[:,(6-num_inputs):6]
raw_data = raw_data.reshape((len(raw_data),1,num_inputs))

In [25]:
raw_data.shape

(96379, 6)

In [10]:
y.shape

(96379,)

In [85]:
#LSTMModel = load_model('./models/25hz/realtime_LSTM_fullActiveData_Tap_32U_6C_epoch_40_loss_0.10.h5')
LSTMModel = load_model('./models/25hz/realtime_LSTM_fullActiveData_Tap_32U_6C_DynamicWpreYMix_epoch_460_loss_0.23.h5')

In [23]:
LSTMModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (1, 1, 3)                 0         
_________________________________________________________________
lstm_1 (LSTM)                [(1, 32), (1, 32), (1, 32 4608      
_________________________________________________________________
dense_1 (Dense)              (1, 1)                    33        
Total params: 4,641
Trainable params: 4,641
Non-trainable params: 0
_________________________________________________________________


In [96]:
pre_y = LSTMModel.predict(raw_data,batch_size=1)

In [97]:
np.savetxt('./TapData/data_preY_v2.csv',pre_y)

In [86]:
weight=LSTMModel.get_weights()

In [20]:
for i in range(len(weight)):
    print(weight[i].shape)

(3, 128)
(32, 128)
(128,)
(32, 1)
(1,)


In [87]:
np.savez('weight6.npz',weight)

In [22]:
weight[0][:,0:32]

array([[-0.0503917 , -0.09018501, -0.10317633, -0.07002152,  0.07048383,
         0.20034751,  0.03185269,  0.14860214,  0.00930676,  0.20367874,
        -0.02039749, -0.05157167,  0.2922875 ,  0.10555371,  0.02545188,
        -0.03003182, -0.19478583, -0.07915115, -0.18803139,  0.27225205,
        -0.04037363, -0.22213322,  0.15193217, -0.1770821 ,  0.17975444,
         0.17992312, -0.07199208, -0.24020289, -0.12940721, -0.21287514,
        -0.33252025,  0.08783754],
       [-0.22590013, -0.08844223,  0.06429048,  0.17536329,  0.10676789,
        -0.11224057,  0.16551955,  0.1938963 , -0.21421365, -0.14478032,
         0.25154886,  0.03621752, -0.3258883 ,  0.12166321,  0.05148561,
         0.08757009,  0.07897988,  0.07529889, -0.14754638,  0.01255301,
         0.16852283, -0.07132989, -0.23259598,  0.17487608, -0.16401118,
         0.15956956, -0.11759657,  0.2691585 ,  0.01861467, -0.29484168,
         0.11111619,  0.03916943],
       [-0.26455644, -0.05323459, -0.34656942,  0.0851

In [14]:
LSTMModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (1, 1, 3)                 0         
_________________________________________________________________
lstm_1 (LSTM)                [(1, 32), (1, 32), (1, 32 4608      
_________________________________________________________________
dense_1 (Dense)              (1, 1)                    33        
Total params: 4,641
Trainable params: 4,641
Non-trainable params: 0
_________________________________________________________________


In [21]:
print(LSTMModel.inputs)

[<tf.Tensor 'input_1:0' shape=(1, 1, 3) dtype=float32>]


In [22]:
print(LSTMModel.outputs)

[<tf.Tensor 'dense_1/Relu:0' shape=(1, 1) dtype=float32>]


In [16]:
def build_model_v2(n_units):
    x_in = Input(batch_shape=(1,1,3))
    h_in = Input(batch_shape=(1,1,n_units))
    c_in = Input(batch_shape=(1,1,n_units))
    lstm_out,h,c =LSTMCell(n_units)(x_in)
    #lstm_out =RNN(lstm_layer,unroll=False) (x_in)
    '''
    h_in = Input(shape=(n_units,))
    c_in = Input(shape=(n_units,))
    lstm = LSTM(n_units, return_state = True)
    lstm_out,lstm_h,lstm_c = lstm(x_in,initial_state =[h_in,c_in])
    '''
    #x = Flatten()(x_in)
    #lstm_con = Concatenate()([lstm_h,lstm_c])
    #lstm_out = Flatten()(x_in)
    y_out = Dense(1,activation = 'tanh')(lstm_out)
    #y_out = Dense(1,activation = 'relu')(y)
    return Model(inputs=x_in,outputs=[y_out,h,c])

In [17]:
realtime_LSTM= build_model_v2(32)

TypeError: call() missing 1 required positional argument: 'states'

In [4]:
realtime_LSTM.save('LSTM_Test2.hdf5')

In [66]:
def build_model_v2(n_units,num_inputs=3):
    x_in = Input(batch_shape=(1,1,num_inputs))
    lstm_out,lstm_h,lstm_c = LSTM(n_units, return_state = True,stateful=True)(x_in)
    '''
    h_in = Input(shape=(n_units,))
    c_in = Input(shape=(n_units,))
    lstm = LSTM(n_units, return_state = True)
    lstm_out,lstm_h,lstm_c = lstm(x_in,initial_state =[h_in,c_in])
    '''
    #x = Flatten()(x_in)
    #lstm_con = Concatenate()([lstm_h,lstm_c])
    
    y_out = Dense(1,activation = 'relu')(lstm_out)
    lstm_state = Concatenate()([lstm_h,lstm_c])
    #y_out = Dense(1,activation = 'relu')(y)
    return Model(inputs=x_in,outputs=[y_out,lstm_state])


In [67]:
rebuildModel=build_model_v2(32)

In [68]:
rebuildModel.layers[2]

In [69]:
rebuildModel.layers[1].set_weights(LSTMModel.layers[1].get_weights())
rebuildModel.layers[2].set_weights(LSTMModel.layers[2].get_weights())

In [78]:
h = rebuildModel.predict(raw_data[0:10],batch_size=1)

In [79]:
h

[array([[-0.],
        [-0.],
        [-0.],
        [-0.],
        [-0.],
        [-0.],
        [-0.],
        [-0.],
        [-0.],
        [-0.]], dtype=float32),
 array([[-0.09490553,  0.01206422,  0.10296744, -0.09917992,  0.07588826,
         -0.07795104,  0.02191551, -0.13767721, -0.04881138, -0.06123666,
          0.05877448,  0.30867592, -0.19316114, -0.29431283, -0.2440839 ,
         -0.04894308, -0.52115715,  0.02771688,  0.03202315,  0.03136631,
         -0.16800883, -0.15039466, -0.08229536, -0.02807628,  0.09979199,
         -0.11563709, -0.03285349, -0.08969718,  0.02577355, -0.2774055 ,
          0.13139752, -0.3810906 , -0.21444742,  0.02582763,  0.2033777 ,
         -0.23192653,  0.16177502, -0.24736127,  0.04738998, -0.21546632,
         -0.08460043, -0.14324403,  0.09372416,  0.52963114, -0.49540657,
         -0.5108493 , -0.5829645 , -0.12721801, -1.3014983 ,  0.07168403,
          0.06727508,  0.06958196, -0.31567806, -0.34007278, -0.20917237,
         -0.0502275

In [82]:
h[1][1]

array([-0.19274455, -0.03347821,  0.09720079, -0.07088806,  0.03686401,
       -0.08647734,  0.00397862, -0.12178545, -0.17076822, -0.09238589,
        0.12401538,  0.27562597, -0.29087672, -0.29242894, -0.27149966,
       -0.04753771, -0.50943685,  0.08064003,  0.04733399,  0.04114906,
       -0.15266436, -0.18348175, -0.13781969, -0.03844652,  0.08821795,
       -0.04088109, -0.08961172, -0.01958807,  0.09929167, -0.21419497,
        0.17613037, -0.38394976, -0.42682523, -0.06820071,  0.17680864,
       -0.14693657,  0.07633375, -0.20108387,  0.00913726, -0.20724332,
       -0.30577448, -0.16377464,  0.22258638,  0.5240675 , -0.5431091 ,
       -0.5409199 , -0.62360895, -0.11270671, -1.202703  ,  0.22548412,
        0.10190035,  0.09684858, -0.34504184, -0.4393366 , -0.3330565 ,
       -0.08439329,  0.17320967, -0.10375077, -0.21333477, -0.04016743,
        0.21967869, -0.42738265,  0.36736208, -0.7918088 ], dtype=float32)